In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.1 MB 7.7 MB/s 
     |████████████████████████████████| 61 kB 476 kB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 8.0 MB/s 
     |████████████████████████████████| 243 kB 47.9 MB/s 
     |████████████████████████████████| 1.1 MB 37.3 MB/s 
     |████████████████████████████████| 132 kB 67.2 MB/s 
     |████████████████████████████████| 271 kB 43.1 MB/s 
     |████████████████████████████████| 192 kB 64.6 MB/s 
     |████████████████████████████████| 160 kB 54.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
# from transformers import AutoModel, BertTokenizerFast, BertModel
from transformers import BertTokenizer
from datasets import load_dataset

In [ ]:
# train_data = pd.read_csv('df_convote_train.csv')
# test_data = pd.read_csv('df_convote_test.csv')

# train_text = train_data.text.values
# train_labels = train_data.political_party.values
# test_text = test_data.text.values
# test_labels = test_data.political_party.values

# train_text, val_text, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.2, random_state=42)
# # train/val/test - text/labels
# train_text

raw_datasets = load_dataset('csv', data_files={'train': 'df_convote_train.csv',
                                               'test' : 'df_convote_test.csv'})
raw_datasets = raw_datasets.remove_columns(['Unnamed: 0', 'bill_debate_number', 'mentioned_bill', 'voted_yes'])
raw_datasets


Using custom data configuration default-7d92425298cbf2a0


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7d92425298cbf2a0/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'labels'],
        num_rows: 5660
    })
    test: Dataset({
        features: ['sentence', 'labels'],
        num_rows: 1759
    })
})

In [ ]:
# Preprocessing
def text_preprocessing(sample):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    - remove references to other speakers (Kevin 12/2)
    """
    sent = sample['sentence']

    # Remove '@name'
    sent = re.sub(r'(@.*?)[\s]', ' ', sent)

    # Replace '&amp;' with '&'
    sent = re.sub(r'&amp;', '&', sent)

    # Remove trailing whitespace
    sent = re.sub(r'\s+', ' ', sent).strip()

    # Remove xz12031 refs from text
    sent = re.sub(r'xz[0-9]+', '', sent)

    sample['text'] = sent

preprocessed_datasets = raw_datasets.map(text_preprocessing)

  0%|          | 0/5660 [00:00<?, ?ex/s]

  0%|          | 0/1759 [00:00<?, ?ex/s]

In [ ]:
# 
# via https://www.analyticsvidhya.com/blog/2021/06/why-and-how-to-use-bert-for-nlp-text-classification/
# 
model_name = 'bert-base-uncased'

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Get encodings
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True, max_length=512)
    
tokenized_datasets = preprocessed_datasets.map(tokenize_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# Split data into train and test splits
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define metrics function for model evaluation https://huggingface.co/transformers/v3.0.2/training.html
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# !pip install GPUtil

# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache()  

In [ ]:
# docs for bert models https://huggingface.co/docs/transformers/model_doc/bert 
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, TrainingArguments, Trainer

# Evaluate on pretrained model
# followed tutorial at https://huggingface.co/docs/transformers/training
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_attentions=False, output_hidden_states=False)
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)

# test_trainer = dir where checkpoints are saved
training_args = TrainingArguments("test_trainer", logging_steps=50, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=3) #, evaluation_strategy="steps") # or steps

# docs for trainer and trainingArguments (3.5 docs are better/more info) https://huggingface.co/transformers/v3.5.1/main_classes/trainer.html#transformers.TrainingArguments
trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset, compute_metrics=compute_metrics
)

# fine-tune our model
trainer.train()


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

RuntimeError: ignored

In [ ]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')